In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

**Read The Two Imges (L & R)**

In [ ]:
r_gray_img = cv2.cvtColor(cv2.imread(('../input/stereods/r1.png')), cv2.COLOR_BGR2GRAY)
l_gray_img = cv2.cvtColor(cv2.imread(('../input/stereods/l1.png')), cv2.COLOR_BGR2GRAY)


print(r_gray_img.shape)
print(l_gray_img.shape)

**1.1 Block Matching**

In [ ]:
def block_matching_normal(r_img,l_img,metric,window_size):  
    disparity_map=np.zeros(r_img.shape)
    for i in range(r_img.shape[0]):
        for rj in range(r_img.shape[1]):
            diff_list=[]
            for lj in range(l_img.shape[1]):                
                temp_diff = abs(r_img[i][rj]-l_img[i][lj]) if metric=="SAD" else abs((r_img[i][rj]-l_img[i][lj])**2) 
                diff_list.append(temp_diff)
            idx_min=diff_list.index(min(diff_list))
            disparity_map[i][rj] = rj - idx_min    
    return np.array(disparity_map)
    

In [ ]:
kernel_center = ((5-1)/2 , (5-1)/2)
print(kernel_center[0])

In [ ]:
def block_matching(r_img,l_img,metric,w):
    kernel=np.ones((w,w))
    print(kernel)
    kernel_half = int((w-1)/2)
    print(kernel_half)
    kernel_center = (int((w-1)/2) , int((w-1)/2))
    print(kernel_center)
    disparity_map=np.zeros(r_img.shape)
    for i in range(kernel_half,r_img.shape[0]-kernel_half):
        print(i)
        for rj in range(kernel_half,r_img.shape[1]-kernel_half):
            r_window = np.multiply(r_img[(i-kernel_half):(i+kernel_half+1),(rj-kernel_half):(rj+kernel_half+1)],kernel)
            diff_list=[]
            for lj in range(kernel_half,l_img.shape[1]-kernel_half):
                l_window = np.multiply(l_img[(i-kernel_half):(i+kernel_half+1),(lj-kernel_half):(lj+kernel_half+1)],kernel)
                temp_diff = np.sum(np.absolute(np.subtract(r_window,l_window))) if metric=="SAD" else abs((r_img[i][lj]-l_img[i][lj])**2) 
                diff_list.append(temp_diff)
            idx_min=diff_list.index(min(diff_list)) + 2
            disparity_map[i][rj] = rj - idx_min    
    return np.array(disparity_map)
    

**Call Block Matching Fun**

In [ ]:
disaprity_map=block_matching(l_gray_img,r_gray_img,"SAD",9)
disaprity_map.shape

In [ ]:
print(l_gray_img.shape)

**Visualize Disparity Map**

In [ ]:
plt.imshow(disaprity_map,"gray")
plt.title('Disparity')
plt.show()

In [ ]:
# create figure
fig = plt.figure(figsize=(17, 15))

# Adds a subplot at the 1st position
fig.add_subplot(1, 3,1)
plt.imshow(l_gray_img,"gray")
plt.title("Left Image (reference)")
fig.add_subplot(1, 3,2)
plt.imshow(r_gray_img,"gray")
plt.title("Right Image")
fig.add_subplot(1, 3,3)
plt.imshow(disaprity_map,"gray")
plt.title("disaprity map")
  

# cv2

In [ ]:
# imgL = cv2.cvtColor(cv2.imread(('../input/stereods/l1.png')), cv2.COLOR_BGR2GRAY)
# imgR = cv2.cvtColor(cv2.imread(('../input/stereods/r1.png')), cv2.COLOR_BGR2GRAY)
# stereo = cv2.StereoBM_create()
# disparity = stereo.compute(imgL,imgR)

# 1.2 Dynamic programming


In [ ]:
plotted_array=[]

In [ ]:
imageL = l_gray_img
imageR = r_gray_img

height = imageL.shape[0]
width = imageR.shape[1]


disparityMap_Right=[]
disparityMap_Left=[]
alpha = 2
cO = 1


for line in range(height):

    # forward Loop
    costMap = np.zeros((width,width),dtype=np.int16)
    flagArray = np.zeros(width,dtype=np.int16)
     
    for i in range(width):
        for j in range(width):
            matched_cost= ( (imageL[line][i]-imageR[line][j])**2 ) / (alpha**2) 
            if i==0 and j==0:
                costMap[i][j] = matched_cost 
            elif i==0 :
                 costMap[i][j] = costMap[i][j-1] + cO #first row
            elif j==0 :
                 costMap[i][j] =   costMap[i-1][j] +cO # first col  
            else :
                costMap[i][j] = min(costMap[i-1][j-1] + matched_cost , costMap[i-1][j] + cO , costMap[i][j-1] + cO )
    
    
    s = width 
    i = width
    j = width 
    #print(s,i,j)
    minDirectionArray= []
    disparityRow_Right= []
    disparityRow_Left =[]


    #first loop 
    minDirectionArray.append(0) # direction = "diagonal "
    i = i-1
    j = j-1
    disparityRow_Right.append(abs(i-j))
    disparityRow_Left.append(abs(i-j))
    #print(flagArray)
    while (j >= 0 and i>=0) :
            upCost= costMap[i-1][j]
            leftCost = costMap[i][j-1]
            diagonalCost= costMap[i-1][j-1]
            #print(leftCost, diagonalCost,upCost)

            #print("i =", "j=" )
            #print( i , j )

            # Special Cases 
            if(j==0):
                minDirectionArray.append(-2) # direction = "UP "
                i = i-1
            elif(i==0):
                minDirectionArray.append(-1) # direction = "left "
                j = j-1


            # Take the minimum 
            elif (diagonalCost <=leftCost and diagonalCost <=upCost ):
                   # matched 
                   # print("diagonal")
                    minDirectionArray.append(0) # direction = "diagonal "
                    i = i-1
                    j = j-1
                    disparityRow_Right.append(abs(i-j))
                    disparityRow_Left.append(abs(i-j))

            elif(upCost <= leftCost):
                    #right occlusion 
                    minDirectionArray.append(-2)#direction = "UP "
                    i = i-1
                    disparityRow_Left.append(0)

                    #left occlusion
            else : 
                    minDirectionArray.append(-1) # direction = "left "
                    j=j-1
                    disparityRow_Right.append(0)

            s = s-1
    if (line == 100 ):
        plotted_array=minDirectionArray
        print (plotted_array ) 


    disparityRow_Right.reverse()
    disparityRow_Left.reverse()
    minDirectionArray.reverse()  
    disparityMap_Right.append(disparityRow_Right)
    disparityMap_Left.append(disparityRow_Left)

In [ ]:
disparityMap_Right_np =  np.zeros((height,width),dtype=np.int16)

In [ ]:
disparityMap_Left_np =  np.zeros((height,width),dtype=np.int16)

In [ ]:
for i in range (len (disparityMap_Left )):
    for j in range (len (disparityMap_Left[i])):

        disparityMap_Left_np[i][j]= disparityMap_Left[i][j]

In [ ]:
for i in range (len (disparityMap_Right )):
    for j in range (len (disparityMap_Right[i] )):

        disparityMap_Right_np[i][j]= disparityMap_Right[i][j]

In [ ]:
np.savetxt("rightMap.csv", disparityMap_Right_np, delimiter=",")
np.savetxt("leftMap.csv", disparityMap_Left_np, delimiter=",")

In [ ]:
disparityMap_Right_np.shape

In [ ]:
disparityMap_Right_np.shape

In [ ]:
plt.imshow(disparityMap_Left_np,"gray")
plt.title('Disparity of left image ')
plt.show()

In [ ]:
plt.imshow(disparityMap_Right_np,"gray")
plt.title('Disparity of left image ')
plt.show()

In [ ]:
def Draw(directionList) :

    x=np.zeros(len(directionList),dtype=np.int16)

    y=np.zeros(len(directionList),dtype=np.int16)
    for i in range(len ( directionList )):
        if(directionList[i]==-1):
            x[i]=x[i-1]+1
            y[i]=y[i-1]
        elif(directionList[i]==-2):
            y[i]=y[i-1]+1
            x[i]+=x[i-1]
        else :
            x[i]=x[i-1]+1
            y[i]=y[i-1]+1
    return x , y 

In [ ]:
x , y =Draw (plotted_array)

In [ ]:
import matplotlib.pyplot as plt

# plotting the points
plt.plot(x, y)
 
# setting x and y axis range

 
# naming the x axis
plt.xlabel('Left Image')
# naming the y axis
plt.ylabel('Right Image')
 
# giving a title to my graph
plt.title('Direction Map of Row 100')
 
# function to show the plot
plt.show()
        